In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\src')

import pandas as pd
import altair as alt
from altair.expr import datum
import datetime as dti
import locale
from Bruit.Emission import Route
from statistics import mean, harmonic_mean

locale.setlocale(locale.LC_ALL, 'fr_FR')
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

# ExD936  
sur cette voie il va falloir extrapoler des données : 
- sur la première semaines de mesure
- sur un des deux sens pendant la deuxième semaine de mesure

> ## comparaison des données Cerema et BdxMet
il s'agit de comparer les données Bdx Met avec les données Cerema, pour pouvoir extrapoler les valeurs sur la 2ème semaine de mesures acoustiques dans les deux sens

In [3]:
# paramètres 
fichierCerema = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_29-03-2022_au_04-04-2022_Cerema\Mesures_individuelles.xlsx'
fichierBM = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_04-04-2022 au 18-04-2022_CEPV\CSV\Semaine1\Vers_Bordeaux\00030093-20220331123748.CSV'

In [4]:
# import des données et mise en forme
dfCerema = pd.read_excel(fichierCerema, names=['horodatage', 'voie', 'contresens', 'vitesse', 'longueur', 'classe_veh'])
dfBm = pd.read_csv(fichierBM, names=['date', 'heure', 'vitesse', 'longueur', 'div', 'voie', 'reverse'] , skiprows=5)
dfBm['horodatage'] = pd.to_datetime(dfBm.date + ' ' + dfBm.heure, dayfirst=True)
dfCerema['classe_veh'] = dfCerema.longueur.apply(lambda x: 'vl' if x <= 6 else 'pl')
dfBm['classe_veh'] = dfBm.longueur.apply(lambda x: 'vl' if x <= 6 else 'pl')

In [5]:
# Comparaison des données
dfBm30min = dfBm.groupby([pd.Grouper(freq='30min', key='horodatage'), 'classe_veh']).agg({'date': 'count', 'vitesse': 'mean'}).reset_index().rename(
    columns={'date': 'nb_veh'}).sort_values(['horodatage', 'classe_veh'])
dfCerema30min = dfCerema.groupby([pd.Grouper(freq='30min', key='horodatage'), 'classe_veh']).agg({'voie': 'count', 'vitesse': 'mean'}).reset_index().rename(
    columns={'voie': 'nb_veh'}).sort_values(['horodatage', 'classe_veh'])
jointureBmCerema30min = dfBm30min.merge(dfCerema30min, on=['horodatage', 'classe_veh'], suffixes=['_bm', '_cerema'])[2:-2]
# comparaison à la voie
dfBm30minVoie = dfBm.groupby([pd.Grouper(freq='30min', key='horodatage'), 'classe_veh', 'voie']).agg({'date': 'count', 'vitesse': 'mean'}).reset_index().rename(
    columns={'date': 'nb_veh'}).sort_values(['horodatage', 'classe_veh'])
dfCerema30minVoie = dfCerema.groupby([pd.Grouper(freq='30min', key='horodatage'), 'classe_veh', 'voie']).agg({'longueur': 'count', 'vitesse': 'mean'}).reset_index().rename(
    columns={'longueur': 'nb_veh'}).sort_values(['horodatage', 'classe_veh'])
jointureBmCerema30minVoie = dfBm30minVoie.merge(dfCerema30minVoie, on=['horodatage', 'classe_veh', 'voie'], suffixes=['_bm', '_cerema'])[2:-2]
# analyse des différences
dfDiffPc = jointureBmCerema30min.groupby('horodatage').apply(
        lambda x: 1 - (sum(x.nb_veh_bm) / sum(x.nb_veh_cerema))).reset_index().rename(columns={0: 'pc_diff_bm_cerema'})
dfCompEmissionBruit = jointureBmCerema30min.pivot(index=['horodatage'], columns=['classe_veh'], values=['nb_veh_bm', 'nb_veh_cerema',
                                                                                                        'vitesse_bm', 'vitesse_cerema']).reset_index()
dfCompEmissionBruit['lwm_bm'] = dfCompEmissionBruit.apply(lambda x: Route(x[('nb_veh_bm', 'vl')], x[('nb_veh_bm', 'pl')],
                                                                          x[('vitesse_bm', 'vl')], x[('vitesse_bm', 'pl')]).lwm, axis=1)
dfCompEmissionBruit['lwm_cerema'] = dfCompEmissionBruit.apply(lambda x: Route(x[('nb_veh_cerema', 'vl')], x[('nb_veh_cerema', 'pl')],
                                                                              x[('vitesse_cerema', 'vl')], x[('vitesse_cerema', 'pl')]).lwm, axis=1)
dfCompEmissionBruit['diff_emission_bruit'] = abs(dfCompEmissionBruit['lwm_cerema'] - dfCompEmissionBruit['lwm_bm'])
dfCompEmissionBruitForm = dfCompEmissionBruit[['horodatage', 'diff_emission_bruit', 'lwm_bm', 'lwm_cerema']].T.reset_index(drop=True).T.rename(
    columns={0: 'horodatage', 1: 'diff_emission_bruit', 2: 'lwm_bm', 3: 'lwm_cerema'})

In [6]:
# Visualisation
dFGraph = pd.melt(jointureBmCerema30min.rename(columns={'nb_veh_bm': 'Bordeaux Metropole', 'nb_veh_cerema': 'Cerema'}),
                  id_vars=(['horodatage', 'classe_veh']), value_vars=('Bordeaux Metropole', 'Cerema'),
                  var_name='src_mesure')[1:-1]
dFGraphVoie = pd.melt(jointureBmCerema30minVoie.rename(columns={'nb_veh_bm': 'Bordeaux Metropole', 'nb_veh_cerema': 'Cerema'}),
                      id_vars=(['horodatage', 'classe_veh', 'voie']), value_vars=('Bordeaux Metropole', 'Cerema'),
                      var_name='src_mesure')[1:-1]
ListCasVoie = ((2, 'Bordeaux Metropole', 'vl'), (2, 'Bordeaux Metropole', 'pl'), (1, 'Bordeaux Metropole', 'vl'), (1, 'Bordeaux Metropole', 'pl'),
               (2, 'Cerema', 'vl'), (2, 'Cerema', 'pl'), (1, 'Cerema', 'vl'), (1, 'Cerema', 'pl'))
dFGraphVoie = pd.concat([dFGraphVoie.loc[(dFGraphVoie.voie == a[0]) & (dFGraphVoie.src_mesure == a[1]) & (dFGraphVoie.classe_veh == a[2])
                                         ].set_index('horodatage').resample('30min').asfreq().assign(voie=a[0], src_mesure=a[1], classe_veh=a[2]).fillna(0)
                         for a in ListCasVoie]).reset_index()
listChart, listChartMirroir, listEmission, listEmmissionMirroir, listNbVoie = [], [], [], [], []
largeurChart = 400
largeurChartMirroirEmission = 600
formatDate = '%A %d %B'
for dateTest in jointureBmCerema30min.horodatage.dt.strftime('%Y-%m-%d').unique():
    # graph de comparaison TV
    jour = dti.datetime.fromisoformat(dateTest).strftime(formatDate)
    titre = f'Véhicules selon source ; le {jour}'
    graphNbVeh = alt.Chart(dFGraph.loc[(dFGraph.src_mesure.isin(('Bordeaux Metropole', 'Cerema'))) &
                                       (dFGraph.horodatage.dt.date == pd.to_datetime(dateTest))], title=titre, width=largeurChart).mark_line().encode(
        x=alt.X('hoursminutes(horodatage):O', title='heure'),
        y=alt.Y('sum(value):Q', title='Nombre de véhicules'),
        color=alt.Color('src_mesure:N', scale=alt.Scale(scheme='dark2'), legend=alt.Legend(values=['Bordeaux Metropole', 'Cerema'], title="Source des mesures"))
    )
    graphDiffNbVeh = alt.Chart(dfDiffPc.loc[dfDiffPc.horodatage.dt.date == pd.to_datetime(dateTest)]).mark_bar(opacity=0.3, color='gray').encode(
        x=alt.X('hoursminutes(horodatage):O', title='heure'),
        y=alt.Y('pc_diff_bm_cerema:Q', title='Pourcentage de variation', axis=alt.Axis(titleColor='gray')))
    listChart.append(alt.layer(graphNbVeh, graphDiffNbVeh).resolve_scale(y='independent'))

    # graph de comparaison par Voie
    base = alt.Chart(dFGraphVoie.loc[(dFGraphVoie.src_mesure.isin(('Bordeaux Metropole', 'Cerema'))) &
                                     (dFGraphVoie.horodatage.dt.date == pd.to_datetime(dateTest))]).mark_line().encode(
        x=alt.X('hoursminutes(horodatage):O', title='Heure'),
        y=alt.Y('sum(value):Q', title='Nb véhicules'),
        color='src_mesure:N')
    graphNbVehVoie = alt.vconcat()
    for voie in [1, 2]:
        graphNbVehVoie &= base.transform_filter(datum.voie == voie).properties(title = f'le {jour}, voie {voie}')
    listNbVoie.append(graphNbVehVoie)

    # graph de comparaison VL / PL
    titreMirroir = f'Type de véhicules selon source ; le {jour}'
    graphMirroir = alt.Chart(dFGraph.loc[dFGraph.horodatage.dt.date == pd.to_datetime(dateTest)].assign(
        type_veh=dFGraph.src_mesure+'_'+dFGraph.classe_veh, value=dFGraph.apply(
            lambda x: 0-x.value if x.src_mesure == 'Cerema' else x.value, axis=1)), title=titreMirroir, width=largeurChart).mark_bar(
    ).encode(
    x=alt.X('hoursminutes(horodatage):O', title='Heure'),
    y=alt.Y('value:Q', title='Nombre de véhicules', axis=alt.Axis(labelExpr='abs(datum.value)')),
    color=alt.Color('type_veh', title='Type de véhicule', legend=alt.Legend(values=['Bordeaux Metropole_pl','Bordeaux Metropole_vl', 'Cerema_pl', 'Cerema_vl'])))
    listChartMirroir.append(graphMirroir)

    # graph de comparaison Bruit
    titreBruit = ['Émission sonore (vitesse moyenne sur la période) selon source ;', f'le {jour}']
    dfGraphEmission = dfCompEmissionBruitForm.loc[dfCompEmissionBruitForm.horodatage.dt.date == pd.to_datetime(dateTest)]
    GraphEmission = alt.Chart(dfGraphEmission, title=titreBruit, width=largeurChart
                              ).transform_fold(['lwm_bm', 'lwm_cerema'],
                                               ).mark_line(strokeWidth=3).encode(
        x=alt.X('hoursminutes(horodatage):O', title='Heure'),
        y=alt.Y('value:Q', scale=alt.Scale(domain=(min(dfGraphEmission.lwm_bm.min(), dfGraphEmission.lwm_cerema.min()), 
                                                   max(dfGraphEmission.lwm_bm.max(), dfGraphEmission.lwm_cerema.max()))), axis=alt.Axis(tickCount=24),
                title='Niveaux d\'émission (db(A))'),
        color=alt.Color('key:N', title='Source du niveau', scale=alt.Scale(scheme='set1'),legend=alt.Legend(values=['lwm_bm','lwm_cerema'])))
    listEmission.append(GraphEmission)

    # combinaison Mirroir et emission
    titreEmmissionMirroir = ['Débit de VL et PL et émission sonore', f'{jour}']
    listEmmissionMirroir.append(alt.layer(graphMirroir.properties(width=largeurChartMirroirEmission),
                                          GraphEmission.properties(width=largeurChartMirroirEmission)
                                          ).resolve_scale(y='independent').resolve_legend('independent').properties(title=titreEmmissionMirroir))

graphCompCeremaBdxMet = alt.vconcat(*listChart)
graphMirroirComplet = alt.vconcat(*listChartMirroir)
GraphEmissionComplet = alt.vconcat(*listEmission)
graphCompEmissionMirroir = alt.vconcat(*listEmmissionMirroir)
graphCompTotalTrafic = (graphCompCeremaBdxMet | graphCompEmissionMirroir).resolve_legend('independent')
graphCompVoie = alt.hconcat(*listNbVoie)

C:\Users\MARTIN~1.SCH\AppData\Local\Temp/ipykernel_30428/2225665352.py:22: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  (dFGraph.horodatage.dt.date == pd.to_datetime(dateTest))], title=titre, width=largeurChart).mark_line().encode(
C:\Users\MARTIN~1.SCH\AppData\Local\Temp/ipykernel_30428/2225665352.py:27: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  graphDiffNbVeh = alt.Chart(dfDiffPc.loc[dfDiffPc.horodatage.dt.date == pd.to_datetime(dateTest)]).mark_bar(opacity=0.3, color='gray').encode(
C:\Users\MARTIN~1.SCH\AppData\Local\Temp/ipykernel_30428/2225665352.py:34: FutureWarning: Comparison o

In [7]:
graphCompVoie

C:\Users\martin.schoreisz\AppData\Roaming\Python\Python39\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.HConcatChart(...)

In [8]:
alt.layer(graphMirroir.properties(width=largeurChartMirroirEmission),
                                          GraphEmission.properties(width=largeurChartMirroirEmission)
                                          ).resolve_scale(y='independent').resolve_legend('independent').properties(title=titreEmmissionMirroir)

alt.LayerChart(...)

In [9]:
# visalusation de la densité de répartition
chartDensiteComplete = alt.Chart(dfCompEmissionBruitForm, title=["Répartition de la différence de niveau d'émission sonore", "selon la source de données de trafic"],
                                width=400).transform_density(
    'diff_emission_bruit',
    as_=['diff_emission_bruit', 'density'],
).mark_area().encode(
    x=alt.X("diff_emission_bruit:Q", title = "Différence de niveau d'émission (dB(A))"),
    y=alt.Y('density:Q', title="densité"))
chartDensiteHeurePointe = alt.Chart(
dfCompEmissionBruitForm.loc[((dfCompEmissionBruitForm.horodatage.dt.time > dti.time(7, 0, 0)) & (dfCompEmissionBruitForm.horodatage.dt.time < dti.time(9, 30, 0))) |
                            ((dfCompEmissionBruitForm.horodatage.dt.time > dti.time(16, 30, 0)) & (dfCompEmissionBruitForm.horodatage.dt.time < dti.time(19, 0, 0)))]
    , title=["Répartition de la différence de niveau d'émission sonore", 
             "en heure pleine selon la source de données de trafic"], width=400).transform_density(
    'diff_emission_bruit',
    as_=['diff_emission_bruit', 'density'],
).mark_area().encode(
    x=alt.X("diff_emission_bruit:Q", title = "Différence de niveau d'émission (dB(A))"),
    y=alt.Y('density:Q', title="densité"))

In [10]:
chartDensiteComplete | chartDensiteHeurePointe

alt.HConcatChart(...)

<div class="alert alert-block alert-warning"><b>Résultats:</b> Il y a une forte divergence des mesures en tout véhicules entre les données fournies par Bordeaux Métropole et celle mesurées par le Cerea, sur la période de recouvrement. Malheureusement cette période ne couvre que du vendredi 01 au lundi 04, mais les résultats sur les heures de pointes indique des variations pouvant aller jusqu'à 50%.  Cela peut impliquer, en prenant en compte les vitesses moyennes, des variations possible supérieures à 2 dB(A)</div>

> ## extrapolation des données trafic du 21 au 29 mars
à partir des données de Bordeaux Métropole, il s'agit de déterminer si le trafic sur la période du 21 au 29 mars peut etre estimé

>> ### comparaison des volumes de VL/PL et des vitesses
on cherche ici à regrouper les jours présentant des caractéristiques proches, afin de pouvoir simuler un jour moyen

In [11]:
# lecture des données individuelles tout sens confondus
fichierVersBordeauxSemaine1 = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_04-04-2022 au 18-04-2022_CEPV\CSV\Semaine1\Vers_Bordeaux\00030093-20220331123748.CSV'
fichierVersBordeauxSemaine2 = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_04-04-2022 au 18-04-2022_CEPV\CSV\Semaine2\Vers_Bordeaux\00030095-20220407104915.CSV'
fichierVersRocadeSemaine1 = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_04-04-2022 au 18-04-2022_CEPV\CSV\Semaine1\Vers_Rocade\00000033-20220331122920.CSV'
fichierVersRocadeSemaine2 = r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\du_04-04-2022 au 18-04-2022_CEPV\CSV\Semaine2\Vers_Rocade\00000119-20220407104120.csv'
dfBm = pd.concat([pd.read_csv(e, names=['date', 'heure', 'vitesse', 'longueur', 'div', 'voie', 'reverse'], skiprows=5).assign(sens=k) 
 for k, v in {'versBordeaux': [fichierVersBordeauxSemaine1, fichierVersBordeauxSemaine2],
              'versRocade': [fichierVersRocadeSemaine1, fichierVersRocadeSemaine2]}.items() 
 for e in v])
dfBm['horodatage'] = pd.to_datetime(dfBm.date + ' ' + dfBm.heure, dayfirst=True)
dfBm['classe_veh'] = dfBm.longueur.apply(lambda x: 'vl' if x <= 6 else 'pl')

In [12]:
# décomposition en jour et périodes horaires
dfBm30min = dfBm.groupby([pd.Grouper(freq='30min', key='horodatage'), 'classe_veh']).agg({'date': 'count', 'vitesse': 'mean'}).reset_index().rename(
    columns={'date': 'nb_veh'}).sort_values(['horodatage', 'classe_veh'])
dfBm30min['jour'] = dfBm30min.horodatage.dt.strftime('%A %Y-%m-%d')
dfBm30min['temp'] = dfBm30min.horodatage.dt.strftime('%X')

In [13]:
# visualisation
largeur = 800
selection = alt.selection_multi(fields=['jour'])
color = alt.condition(selection,
                      alt.Color('jour:N', legend=None),
                      alt.value('lightgray'))
dfVl = dfBm30min.loc[dfBm30min.classe_veh == 'vl']
dfPl = dfBm30min.loc[dfBm30min.classe_veh == 'pl']
graphTraficVl = alt.Chart(dfVl, width=largeur).mark_line().encode( 
    x='hoursminutes(horodatage):T',
    y='nb_veh:Q',
    color=color)
legendVl = alt.Chart(dfVl).mark_point().encode(
    y=alt.Y('jour:N', axis=alt.Axis(orient='right')),
    color=color).add_selection(
    selection)
graphTraficPl = alt.Chart(dfPl, width=largeur).mark_line().encode( 
    x='hoursminutes(horodatage):T',
    y='nb_veh:Q',
    color=color)
legendPl = alt.Chart(dfPl).mark_point().encode(
    y=alt.Y('jour:N', axis=alt.Axis(orient='right')),
    color=color).add_selection(
    selection)
graphTraficVlVts = alt.Chart(dfVl, width=largeur).mark_line().encode( 
    x='hoursminutes(horodatage):T',
    y='vitesse:Q',
    color=color)
graphTraficPlVts = alt.Chart(dfPl, width=largeur).mark_line().encode( 
    x='hoursminutes(horodatage):T',
    y='vitesse:Q',
    color=color)
alt.vconcat((graphTraficVl | legendVl).properties(title='Répartition des VL'), (graphTraficPl | legendPl).properties(title='Répartition des PL'))

C:\Users\martin.schoreisz\AppData\Roaming\Python\Python39\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [14]:
alt.vconcat((graphTraficVlVts | legendVl).properties(title='Vitesses des VL'), (graphTraficPlVts | legendPl).properties(title='Vitesses des PL'))

alt.VConcatChart(...)

***Il ressort de l'analyse des trafics que :*** 
- les lundi 04 et 11/04, mardi 05 et 12/04, mercredi 06 et 13/04, jeudi 07 et 14/04 et vendredi 01 et 08/04 présentent des débits VL et PL proches.
- le week-end du 15 au 19 avril (édbut des vacances scolaires) présente des valeusr de trafics souvent décorrélées du reste des mesures
- le 31/03 et le 19/04 sont les jours de pose et dépose, et ne présentent pas de jours complets
- les samedi 02 et 09/04 présentent des valeurs fortement concordantes
- les dimanches 03 et 10/04 présentent une variation notable
- les vitesses VL et PL varient peu, seules les périodes heures de pointe du matin et du soir varient d'environ 5 km/h entre les jours ouvrés conservés précédemment (cf point 1 de la liste) et les WE

In [15]:
# préparation des données pour estimation de jours ouvrés, samedi et dimanche moyen
dfBm30minJO = dfBm30min.loc[dfBm30min.horodatage.dt.dayofweek.isin(range(5))].copy()
dfBm30minSam = dfBm30min.loc[dfBm30min.horodatage.dt.dayofweek == 5].copy()
dfBm30minDim = dfBm30min.loc[dfBm30min.horodatage.dt.dayofweek == 6].copy()

In [39]:
# calcul des valeurs moyennes
dfBm30minJO_moy = dfBm30minJO.groupby(['temp', 'classe_veh']).agg({'nb_veh': 'median', 'vitesse': lambda x: harmonic_mean(list(x))}
                                                                  ).reset_index().assign(type_jour='ouvre')
dfBm30minSam_moy = dfBm30minSam.groupby(['temp', 'classe_veh']).agg({'nb_veh': 'mean', 'vitesse': lambda x: harmonic_mean(list(x))}
                                                                    ).reset_index().assign(type_jour='samedi')
dfBm30minDim_moy = dfBm30minDim.groupby(['temp', 'classe_veh']).agg({'nb_veh': 'mean', 'vitesse': lambda x: harmonic_mean(list(x))}
                                                                    ).reset_index().assign(type_jour='dimanche')
dfBm30minMoy = pd.concat([dfBm30minJO_moy, dfBm30minSam_moy, dfBm30minDim_moy])
dfBm30minMoy['type_jour'] = dfBm30minMoy['type_jour']+'_moy'

In [40]:
# visualisation
dfVlMoy = pd.concat([dfBm30minMoy.loc[dfBm30minMoy.classe_veh == 'vl'].rename(columns={'type_jour': 'jour'}), dfVl])
dfVlMoy['taille'] = dfVlMoy.jour.apply(lambda x: 1 if not x[-3:] == 'moy' else 10) 
dfPlMoy = pd.concat([dfBm30minMoy.loc[dfBm30minMoy.classe_veh == 'pl'].rename(columns={'type_jour': 'jour'}), dfPl])
dfPlMoy['taille'] = dfPlMoy.jour.apply(lambda x: 1 if not x[-3:] == 'moy' else 10) 
graphTraficVlMoy = alt.Chart(dfVlMoy, width=largeur).mark_line().encode( 
    x='temp:N',
    y='nb_veh:Q',
    color=color,
    size=alt.Size('taille:Q', scale=None))
legendVlMoy = alt.Chart(dfVlMoy).mark_point().encode(
    y=alt.Y('jour:N', axis=alt.Axis(orient='right')),
    color=color).add_selection(
    selection)
graphTraficPlMoy = alt.Chart(dfPlMoy, width=largeur).mark_line().encode( 
    x='temp:N',
    y='nb_veh:Q',
    color=color,
    size=alt.Size('taille:Q', scale=None))
legendPlMoy = alt.Chart(dfPlMoy).mark_point().encode(
    y=alt.Y('jour:N', axis=alt.Axis(orient='right')),
    color=color).add_selection(
    selection)
graphTraficVlVtsMoy = alt.Chart(dfVlMoy, width=largeur).mark_line().encode( 
    x='temp:N',
    y='vitesse:Q',
    color=color,
    size=alt.Size('taille:Q', scale=None))
graphTraficPlVtsMoy = alt.Chart(dfPlMoy, width=largeur).mark_line().encode( 
    x='temp:N',
    y='vitesse:Q',
    color=color,
    size=alt.Size('taille:Q', scale=None))
alt.vconcat((graphTraficVlMoy | legendVlMoy).properties(title='Répartition des VL'), (graphTraficPlMoy | legendPlMoy).properties(title='Répartition des PL'))

C:\Users\martin.schoreisz\AppData\Roaming\Python\Python39\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.VConcatChart(...)

In [18]:
alt.vconcat((graphTraficVlVtsMoy | legendVlMoy).properties(title='Vitesses des VL'), (graphTraficPlVtsMoy | legendPlMoy).properties(title='Vitesses des PL'))

alt.VConcatChart(...)

>> ### comparaison des Niveau d'émission
après détermination des volumns et vitesses moyenne, on va comparer les niveaux de bruit obtenus avec le min et max

In [42]:
# calcul des emissions
def type_jour(jourInt):
    if jourInt in (0, 1, 2, 3, 4):
        return 'ouvre'
    elif jourInt == 5:
        return 'samedi'
    else:
        return 'dimanche'


datesRef = pd.date_range('2022-04-01', '2022-04-14')
dfBm30minEmission = dfBm30min.loc[dfBm30min.horodatage.dt.date.isin([e.date() for e in pd.date_range('2022-04-01', '2022-04-14')])].assign(type_jour=dfBm30min.horodatage.apply(lambda x: type_jour(x.dayofweek)))
dfBm30minEmission = dfBm30minEmission.pivot(index=['jour', 'temp', 'type_jour'], columns=['classe_veh'], values=['nb_veh', 'vitesse']).reset_index()
dfBm30minEmission['lwm'] = dfBm30minEmission.apply(lambda x: Route(x[('nb_veh', 'vl')], x[('nb_veh', 'pl')],
                                                                          x[('vitesse', 'vl')], x[('vitesse', 'pl')], ignoreErreurVts=True).lwm, axis=1)
dfBm30minEmission = dfBm30minEmission.T.reset_index(drop=True).T.rename(columns={0: 'jour', 1: 'temp', 2: 'type_jour', 3: 'nb_veh_pl', 4: 'nb_veh_vl', 
                                                             5: 'vitesse_pl', 6: 'vitesse_vl', 7: 'lwm'})
dfCompEmissionBruitMoy = dfBm30minMoy.pivot(index=['temp', 'type_jour'], columns=['classe_veh'], values=['nb_veh', 'vitesse']).reset_index()
dfCompEmissionBruitMoy['lwm_moy'] = dfCompEmissionBruitMoy.apply(lambda x: Route(x[('nb_veh', 'vl')], x[('nb_veh', 'pl')],
                                                                          x[('vitesse', 'vl')], x[('vitesse', 'pl')], ignoreErreurVts=True).lwm, axis=1)
dfCompEmissionBruitMoy = dfCompEmissionBruitMoy.T.reset_index(drop=True).T.rename(columns={0: 'temp', 1: 'type_jour', 2: 'nb_veh_pl_moy', 3: 'nb_veh_vl_moy', 
                                                             4: 'vitesse_pl_moy', 5: 'vitesse_vl_moy', 6: 'lwm_moy'})
dfCompEmissionBruitMoy['type_jour'] = dfCompEmissionBruitMoy.type_jour.str[:-4]
dfBm30minEmissionMinMax = dfBm30minEmission.groupby(['temp', 'type_jour']).agg(lwm_min=pd.NamedAgg(column="lwm", aggfunc="min"),
                                                     lwm_max=pd.NamedAgg(column="lwm", aggfunc="max")).reset_index()
CompEmissionsMoy = dfCompEmissionBruitMoy.merge(dfBm30minEmissionMinMax, on=['temp', 'type_jour'])
CompEmissionsMoy['ecart_maxi'] = CompEmissionsMoy.apply(lambda x: x.lwm_moy-x.lwm_min if abs(x.lwm_moy-x.lwm_min) > abs(x.lwm_moy-x.lwm_max) 
                                                        else x.lwm_moy-x.lwm_max, axis=1)

In [52]:
CompEmissionsMoy.loc[CompEmissionsMoy.type_jour == 'samedi']#.head(5)

,temp,type_jour,nb_veh_pl_moy,nb_veh_vl_moy,vitesse_pl_moy,vitesse_vl_moy,lwm_moy,lwm_min,lwm_max,ecart_maxi
2,00:00:00,samedi,2.666667,244.333333,55.849163,60.056718,76.5,75.79,76.75,0.71
5,00:30:00,samedi,2.333333,196.333333,66.632723,61.124037,75.82,75.12,76.25,0.70
8,01:00:00,samedi,5.333333,160.0,52.875733,61.585671,75.35,74.63,75.42,0.72
11,01:30:00,samedi,2.0,128.0,51.881722,62.290712,74.04,73.58,74.73,-0.69
14,02:00:00,samedi,1.5,116.0,71.076923,62.362484,73.78,73.33,73.72,0.45
17,02:30:00,samedi,1.0,93.666667,42.0,63.959904,72.71,71.45,73.43,1.26
20,03:00:00,samedi,1.666667,72.0,60.489796,62.202861,71.86,70.67,72.57,1.19
23,03:30:00,samedi,1.666667,66.0,52.608075,62.644917,71.44,71.68,72.05,-0.61
26,04:00:00,samedi,1.666667,60.0,63.177469,63.162519,71.35,70.75,71.10,0.60
29,04:30:00,samedi,2.666667,69.0,52.873814,65.106547,72.18,71.57,72.34,0.61


In [50]:
CompEmissionsMoy

,temp,type_jour,nb_veh_pl_moy,nb_veh_vl_moy,vitesse_pl_moy,vitesse_vl_moy,lwm_moy,lwm_min,lwm_max,ecart_maxi
0,00:00:00,dimanche,3.333333,248.666667,62.607588,58.856678,76.58,77.32,77.44,-0.86
1,00:00:00,ouvre,3.0,110.0,56.477756,61.639071,73.66,71.85,75.38,1.81
2,00:00:00,samedi,2.666667,244.333333,55.849163,60.056718,76.5,75.79,76.75,0.71
3,00:30:00,dimanche,4.5,210.333333,60.119701,60.411833,76.25,76.64,77.29,-1.04
4,00:30:00,ouvre,2.0,81.0,54.875143,62.984844,72.39,69.76,73.93,2.63
...,...,...,...,...,...,...,...,...,...,...
139,23:00:00,ouvre,7.5,270.0,56.869929,59.27566,77.32,74.65,79.39,2.67
140,23:00:00,samedi,7.0,302.0,57.235823,59.309531,77.7,78.38,78.39,-0.69
141,23:30:00,dimanche,2.666667,132.0,57.653593,60.548122,74.17,73.76,75.19,-1.02
142,23:30:00,ouvre,2.0,205.0,54.704658,60.040143,75.69,72.50,79.14,-3.45


In [43]:
largeur=600
((alt.Chart(
    CompEmissionsMoy.loc[CompEmissionsMoy.type_jour == 'ouvre'], width=largeur).mark_area(opacity=0.3).encode(
    x=alt.X('temp', title='heure', axis=alt.Axis(labelOverlap=True)),
    y=alt.Y('lwm_min', scale=alt.Scale(domain=(60,90)), axis=alt.Axis(tickCount=12), title="Niveau d'émission (dB(A))"),  # 
    y2='lwm_max') + alt.Chart(
CompEmissionsMoy.loc[CompEmissionsMoy.type_jour == 'ouvre']).mark_line().encode(
    x='temp',
    y=alt.Y('lwm_moy', scale=alt.Scale(domain=(60,90)), axis=alt.Axis(tickCount=0, labels=False), title=None))).resolve_scale(y='shared') + 
alt.Chart(
CompEmissionsMoy.loc[CompEmissionsMoy.type_jour == 'ouvre']).mark_line(color='green').encode(
    x='temp',
    y=alt.Y('ecart_maxi', scale=alt.Scale(domain=(-5,5)), axis=alt.Axis(labelColor='green', titleColor='green', tickColor='green'))) + 
 alt.Chart(CompEmissionsMoy.loc[CompEmissionsMoy.type_jour == 'ouvre'].assign(y1=1, y2=-1)).mark_area(opacity=0.2, color='green').encode(
 x='temp',
 y=alt.Y('y1:Q', scale=alt.Scale(domain=(-5,5)), axis=alt.Axis(labels=False, title=None)),
 y2='y2:Q')).resolve_scale(
    y='independent').properties(title=["Variation du niveau d'émission moyen estimé par rapports aux niveaux minimum et maximum constatés ;", 'Jours Ouvrés'])

C:\Users\martin.schoreisz\AppData\Roaming\Python\Python39\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.LayerChart(...)

<div class="alert alert-block alert-warning"><b>Résultats:</b> Bien qu'il y ai une divergence notable des niveaux d'émissions entre les valeurs mesurées et les valeurs estimées sur la période 7h00 - 9h00, le reste de la période journée présente des résultats suffisamment concordant avec les mesures pour que les estimations soient validées</div>

>> ### Débits et vitesse VL et PL estimés
mise en forme et export des données

In [44]:
CompEmissionsMoy[['temp', 'type_jour', 'nb_veh_pl_moy', 'nb_veh_vl_moy', 'vitesse_pl_moy', 'vitesse_vl_moy']].rename(columns={
    'nb_veh_pl_moy': 'pl', 'nb_veh_vl_moy': 'vl', 'vitesse_pl_moy': 'vts_pl', 'vitesse_vl_moy': 'vts_vl'
}).to_json(r'C:\Users\martin.schoreisz\Box\Cerema\3E-Prod_Env_Risq\E6-Nuisances\E61-Acoustique\60-Etudes_Locales\2022-BruitRessentiBdxMet\4-Travaux_en_cours\mesures\trafic\exD936\estimation_Q_V\estimationDebitsVitesse.json')